<a href="https://colab.research.google.com/github/qasimzee/isic2024/blob/main/isic_2024_challenge_table_voting_class.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [65]:
!pip install scikit-learn==1.2.2
!pip install numpy==1.26.4

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import VotingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
import xgboost as xgb


In [66]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [67]:
!ls "/content/drive/MyDrive/kaggle/isic-2024-data"

cnn.pkl			 sample_submission.csv	test-metadata.csv	  train-metadata-v1.csv
logistic_regression.pkl  scaler.pkl		test-metadata.csv.gsheet
preprocessor.pkl	 sequential_dnn.pkl	train-image.hdf5
random_forest_model.pkl  test-image.hdf5	train-metadata.csv


In [68]:
METADATA_FILE = "/content/drive/MyDrive/kaggle/isic-2024-data/train-metadata.csv"
metadata_df = pd.read_csv(METADATA_FILE)


<ipython-input-68-6d834c4c6865>:2: DtypeWarning: Columns (51,52) have mixed types. Specify dtype option on import or set low_memory=False.
  metadata_df = pd.read_csv(METADATA_FILE)


In [69]:
TESTDATA_FILE = "/content/drive/MyDrive/kaggle/isic-2024-data/test-metadata.csv"
test_data_df = pd.read_csv(TESTDATA_FILE)
test_data_df

,isic_id,patient_id,target,age_approx,sex,anatom_site_general,clin_size_long_diam_mm,image_type,tbp_tile_type,tbp_lv_A,...,tbp_lv_radial_color_std_max,tbp_lv_stdL,tbp_lv_stdLExt,tbp_lv_symm_2axis,tbp_lv_symm_2axis_angle,tbp_lv_x,tbp_lv_y,tbp_lv_z,attribution,copyright_license
0,ISIC_0015657,IP_6074337,1,45,male,posterior torso,2.70,TBP tile: close-up,3D: XP,22.80433,...,0.304827,1.281532,2.299935,0.479339,20,-155.06510,1511.222000,113.980100,Memorial Sloan Kettering Cancer Center,CC-BY
1,ISIC_0015729,IP_1664139,0,35,female,lower extremity,2.52,TBP tile: close-up,3D: XP,16.64867,...,0.000000,1.271940,2.011223,0.426230,25,-112.36924,629.535889,-15.019287,"Frazer Institute, The University of Queensland...",CC-BY
2,ISIC_0015740,IP_7142616,1,65,male,posterior torso,3.16,TBP tile: close-up,3D: XP,24.25384,...,0.230742,1.080308,2.705857,0.366071,110,-84.29282,1303.978000,-28.576050,FNQH Cairns,CC-BY


In [70]:
train_df, test_df = train_test_split(metadata_df, test_size=0.2, random_state=42)

Y_test = test_df['target']


In [71]:
Y = train_df["target"]

dropped_columns = ["isic_id", "target", "patient_id", "lesion_id", "iddx_full", "iddx_1", "iddx_2", "iddx_3", "iddx_4", "iddx_5", "mel_mitotic_index", "mel_thick_mm", "tbp_lv_dnn_lesion_confidence"]
X = train_df.drop(columns=dropped_columns)

# Replace NaN values
X['sex'] = X['sex'].replace(pd.NA, 'unknown')
X['anatom_site_general'] = X['anatom_site_general'].replace(pd.NA, 'unknown')

X.fillna(0, inplace=True)

In [72]:
# Identify columns with mixed types

for column in X.columns:
  if X[column].apply(type).nunique() > 1:
    print(f"{column}: {X[column].apply(type).unique()}")


In [73]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Identify categorical features
categorical_columns = X.select_dtypes(include=['object']).columns
numerical_columns = X.select_dtypes(include=['number']).columns

# Create a ColumnTransformer with OneHotEncoder
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(handle_unknown='ignore', sparse_output=False), categorical_columns),
        ('num', StandardScaler(), numerical_columns)
    ],
    remainder='passthrough'
)

X_train_encoded = preprocessor.fit_transform(X)


# Assuming you've already preprocessed your categorical variables
model = Sequential([
    Dense(64, input_shape=(X_train_encoded.shape[1],), activation='relu'),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(X_train_encoded, Y, epochs=20, batch_size=32)



Epoch 1/20


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


10027/10027 ━━━━━━━━━━━━━━━━━━━━ 18s 2ms/step - accuracy: 0.9954 - loss: 0.0161
Epoch 2/20
10027/10027 ━━━━━━━━━━━━━━━━━━━━ 17s 2ms/step - accuracy: 0.9990 - loss: 0.0059
Epoch 3/20
10027/10027 ━━━━━━━━━━━━━━━━━━━━ 20s 2ms/step - accuracy: 0.9991 - loss: 0.0054
Epoch 4/20
10027/10027 ━━━━━━━━━━━━━━━━━━━━ 17s 2ms/step - accuracy: 0.9990 - loss: 0.0053
Epoch 5/20
10027/10027 ━━━━━━━━━━━━━━━━━━━━ 18s 2ms/step - accuracy: 0.9990 - loss: 0.0051
Epoch 6/20
10027/10027 ━━━━━━━━━━━━━━━━━━━━ 18s 2ms/step - accuracy: 0.9989 - loss: 0.0057
Epoch 7/20
10027/10027 ━━━━━━━━━━━━━━━━━━━━ 20s 2ms/step - accuracy: 0.9990 - loss: 0.0051
Epoch 8/20
10027/10027 ━━━━━━━━━━━━━━━━━━━━ 30s 3ms/step - accuracy: 0.9989 - loss: 0.0053
Epoch 9/20
10027/10027 ━━━━━━━━━━━━━━━━━━━━ 27s 2ms/step - accuracy: 0.9991 - loss: 0.0046
Epoch 10/20
10027/10027 ━━━━━━━━━━━━━━━━━━━━ 21s 2ms/step - accuracy: 0.9990 - loss: 0.0050
Epoch 11/20
10027/10027 ━━━━━━━━━━━━━━━━━━━━ 17s 2ms/step - accuracy: 0.9990 - loss: 0.0044
Epoch 12

In [74]:
# test_df['sex'] = test_df['sex'].replace(pd.NA, 'unknown')
# test_df['anatom_site_general'] = test_df['anatom_site_general'].replace(pd.NA, 'unknown')
# test_df['lesion_id'] = test_df['lesion_id'].replace(pd.NA, 'unknown')
# test_df['iddx_2'] = test_df['iddx_2'].replace(pd.NA, 'unknown')
# test_df['iddx_3'] = test_df['iddx_3'].replace(pd.NA, 'unknown')
# test_df['iddx_4'] = test_df['iddx_4'].replace(pd.NA, 'unknown')
# test_df['mel_mitotic_index'] = test_df['mel_mitotic_index'].replace(pd.NA, 'unknown')

# test_df.fillna(0, inplace=True)
# # Identify columns with mixed types
# for column in test_df.columns:
#   if test_df[column].apply(type).nunique() > 1:
#     print(f"{column}: {test_df[column].unique()}")




In [75]:
test_isic_ids = test_df['isic_id']
test_df = test_df.drop(columns=dropped_columns)
test_df.head()

,age_approx,sex,anatom_site_general,clin_size_long_diam_mm,image_type,tbp_tile_type,tbp_lv_A,tbp_lv_Aext,tbp_lv_B,tbp_lv_Bext,...,tbp_lv_radial_color_std_max,tbp_lv_stdL,tbp_lv_stdLExt,tbp_lv_symm_2axis,tbp_lv_symm_2axis_angle,tbp_lv_x,tbp_lv_y,tbp_lv_z,attribution,copyright_license
278442,55.0,female,lower extremity,3.15,TBP tile: close-up,3D: XP,15.998595,8.383633,28.868481,25.256029,...,1.007245,3.890118,1.672431,0.136691,140,160.508423,354.662811,72.125671,ACEMID MIA,CC-0
215021,40.0,male,posterior torso,3.03,TBP tile: close-up,3D: XP,17.651690,12.653870,31.869840,29.712320,...,0.987079,2.747920,2.701037,0.264865,150,144.663000,1240.713000,94.129270,"Department of Dermatology, Hospital Clínic de ...",CC-BY-NC
209685,85.0,male,lower extremity,2.72,TBP tile: close-up,3D: XP,11.409180,9.934399,16.911940,19.329230,...,1.088047,1.835973,2.095205,0.239726,130,-290.056300,103.316300,63.942630,"ViDIR Group, Department of Dermatology, Medica...",CC-BY-NC
29648,55.0,male,posterior torso,3.58,TBP tile: close-up,3D: XP,15.982470,12.767900,22.729350,21.831640,...,0.592206,1.047462,1.364019,0.287234,135,-81.866640,1614.268000,72.240660,"ViDIR Group, Department of Dermatology, Medica...",CC-BY-NC
323386,70.0,male,posterior torso,5.52,TBP tile: close-up,3D: XP,13.933450,7.950915,16.302050,17.114270,...,2.765024,2.548177,2.200729,0.460039,130,80.211790,1107.658000,153.928100,"Department of Dermatology, Hospital Clínic de ...",CC-BY-NC


In [76]:
# Columns in test_data_df but not in test_df
diff1 = set(test_data_df.columns) - set(test_df.columns)
print("Columns in test_data_df but not in test_df:", diff1)

# Columns in test_df but not in test_data_df
diff2 = set(test_df.columns) - set(test_data_df.columns)
print("Columns in test_df but not in test_data_df:", diff2)
test_data_isic_ids = test_data_df['isic_id']
test_data_df = test_data_df.drop(columns=['target', 'isic_id', 'patient_id'])

print(test_data_df.shape)
print(test_df.shape)


Columns in test_data_df but not in test_df: {'target', 'patient_id', 'isic_id'}
Columns in test_df but not in test_data_df: set()
(3, 42)
(80212, 42)


In [77]:
encoder = preprocessor.named_transformers_['cat']
print("Categories learned by OneHotEncoder:", encoder.categories_)

Categories learned by OneHotEncoder: [array(['female', 'male', 'unknown'], dtype=object), array(['anterior torso', 'head/neck', 'lower extremity',
       'posterior torso', 'unknown', 'upper extremity'], dtype=object), array(['TBP tile: close-up'], dtype=object), array(['3D: XP', '3D: white'], dtype=object), array(['Head & Neck', 'Left Arm', 'Left Arm - Lower', 'Left Arm - Upper',
       'Left Leg', 'Left Leg - Lower', 'Left Leg - Upper', 'Right Arm',
       'Right Arm - Lower', 'Right Arm - Upper', 'Right Leg',
       'Right Leg - Lower', 'Right Leg - Upper', 'Torso Back',
       'Torso Back Bottom Third', 'Torso Back Middle Third',
       'Torso Back Top Third', 'Torso Front', 'Torso Front Bottom Half',
       'Torso Front Top Half', 'Unknown'], dtype=object), array(['Head & Neck', 'Left Arm', 'Left Leg', 'Right Arm', 'Right Leg',
       'Torso Back', 'Torso Front', 'Unknown'], dtype=object), array(['ACEMID MIA',
       'Department of Dermatology, Hospital Clínic de Barcelona',
     

In [78]:
# Make predictions on the test data

X_test_encoded = preprocessor.transform(test_df)

# X_test_encoded = preprocessor.transform(test_data_df)
print(X_test_encoded.shape)
y_pred_proba = model.predict(X_test_encoded)  # Probability of positive class (e.g., malignant)

# Create a DataFrame to map predictions back to 'isic_id'
predictions_df = pd.DataFrame({
    'isic_id': test_isic_ids,
    'target': y_pred_proba.flatten()
})


predictions_df



(80212, 85)
2507/2507 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step


,isic_id,target
278442,ISIC_6973879,6.661222e-08
215021,ISIC_5407194,1.321130e-07
209685,ISIC_5273739,8.401832e-06
29648,ISIC_0802250,1.179110e-08
323386,ISIC_8084953,6.792283e-07
...,...,...
2741,ISIC_0139843,1.652755e-06
239797,ISIC_6027282,9.641648e-06
243397,ISIC_6115419,1.064054e-07
86684,ISIC_2218691,1.675593e-05


In [79]:
y_pred_proba = np.nan_to_num(y_pred_proba, nan=0)


In [80]:
(y_pred_proba > 0.5).sum()

14

In [81]:
import numpy as np
from sklearn.metrics import roc_curve, auc

# Compute the ROC curve
fpr, tpr, thresholds = roc_curve(Y_test, y_pred_proba)

# Specify the range of TPR you are interested in (e.g., above 80%)
# Filter the points where TPR is above 0.8
tpr_threshold = 0.8
valid_range = tpr >= tpr_threshold

# Compute pAUC within the valid range
partial_auc = auc(fpr[valid_range], tpr[valid_range])

print(f"Partial AUC for TPR > 80%: {partial_auc:.2f}")


Partial AUC for TPR > 80%: 0.84


In [82]:
import pickle
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import LogisticRegression

# Assume you have trained your pipeline as described before
with open('/content/drive/MyDrive/kaggle/isic-2024-data/preprocessor.pkl', 'wb') as f:
    pickle.dump(preprocessor, f)

# Save the entire pipeline to a pickle file
with open('/content/drive/MyDrive/kaggle/isic-2024-data/sequential_dnn.pkl', 'wb') as file:
    pickle.dump(model, file)

print("Model pipeline saved successfully!")


Model pipeline saved successfully!
